In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from darts import TimeSeries
from darts.models import NLinearModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics import MetricCollection, MeanAbsoluteError, MeanSquaredError
import torch

torch.set_float32_matmul_precision('high')

In [2]:
# base_path = 'sample_csv/'
base_path = 'stock_data_kr_csv/'

column_mapping = {
    '일자': 'date',
    '종가': 'closing_price',
    '대비': 'price_difference',
    '등락률': 'fluctuation_rate',
    '시가': 'opening_price',
    '고가': 'highest_price',
    '저가': 'lowest_price',
    '거래량': 'trading_volume',
    '거래대금': 'trading_value',
    '시가총액': 'market_capitalization',
    '상장주식수': 'listed_shares',
}

In [3]:
stock_df = pd.read_csv(os.path.join(base_path, '005930_삼성전자_20000101_20230331.csv')).rename(columns=column_mapping)

stock_df.head()

,date,closing_price,price_difference,fluctuation_rate,opening_price,highest_price,lowest_price,trading_volume,trading_value,market_capitalization,listed_shares
0,2023/03/31,64000,800,1.27,64000,64000,63700,14094479,900711556583,382066083200000,5969782550
1,2023/03/30,63200,500,0.80,63700,63700,63100,15684377,993903387442,377290257160000,5969782550
2,2023/03/29,62700,-200,-0.32,62500,62700,62200,11216008,700455977590,374305365885000,5969782550
3,2023/03/28,62900,800,1.29,62400,62900,62100,11614118,726295104112,375499322395000,5969782550
4,2023/03/27,62100,-900,-1.43,62600,62800,62000,11039331,687170094400,370723496355000,5969782550


과거 공변량:
<ol>
    <li>과거 주가 데이터: 종가, 시가, 고가, 저가 등
    <li>거래량: 주식 거래량 데이터
    <li>기술적 지표: 이동 평균, RSI (상대 강도 지수), MACD (이동 평균 수렴 발산) 등
    <li>기업 재무 데이터: 매출, 이익, 자산, 부채 등
    <li>배당: 배당금과 배당 수익률
    <li>주가 수익률 (P/E) 등의 가치 지표
</ol>
<br>
미래 공변량:
<ol>
    <li>경제 성장률: 국내 총생산(GDP) 성장률 등
    <li>인플레이션: 소비자 물가지수(CPI) 등
    <li>기준 금리: 중앙은행의 기준 금리
    <li>실업률: 국가의 실업률 데이터
    <li>환율: 외환 시장의 환율 정보
    <li>경제 정책: 정부의 경제 정책이나 중앙은행의 통화 정책
</ol>

In [10]:
# past_cov_list = ['price_difference', 'fluctuation_rate', 'opening_price', 'highest_price', 'lowest_price', 'trading_volume',
#                 'trading_value', 'market_capitalization', 'listed_shares']
past_cov_list = []

def getScaledPastCov(timeseries):
    past_cov_timeseries = timeseries.copy()
    cov_year = datetime_attribute_timeseries(past_cov_timeseries, attribute='year')
    cov_month = datetime_attribute_timeseries(past_cov_timeseries, attribute='month')
    past_cov = cov_year.stack(cov_month)
    for col in past_cov_list:
        past_cov = past_cov.stack(past_cov_timeseries[col])
    return Scaler().fit_transform(past_cov)

def getFutureCov(timeseries):
    pass

In [57]:
"""
    아래 코드의 속도 개선 중
"""
import time
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')


start_date = '2015-01-01'
end_date = '2023-03-31'

start_time = time.time()

for i, file_name in enumerate(os.listdir(base_path)):
    print(i, file_name)
    df = pd.read_csv(base_path + file_name).rename(columns=column_mapping)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    if len(df.loc[:start_date]) == 0: # start_date 이전 날짜 데이터가 없는 경우는 무시. 따라서 start_date 이후 상장된 주식은 훈련 X
        continue
    df = df.loc[start_date:end_date]
    # 9.68초
    
    # Date interpolate
    stock_timeseries_df_interpolated = df.copy()
    stock_timeseries_df_interpolated = stock_timeseries_df_interpolated.asfreq('B')
    for col in interpolate_list:
        stock_timeseries_df_interpolated[col] = stock_timeseries_df_interpolated[col].interpolate(method='linear')
    df = stock_timeseries_df_interpolated
    # 25.73초
    
    # 전처리 결과를 저장
#     df.to_excel('interpolated_' + base_path + file_name)
    
#     timeseries = TimeSeries.from_dataframe(df)
    # 40.93초
    
    
    # Darts TimeSeries 변환 및 날짜 필터링
#     stock_timeseries = timeseries['closing_price']
#     list_stock_timeseries.append(stock_timeseries)

    # Past covariant 및 날짜 필터링
#     scaled_past_cov_timeseries = getScaledPastCov(timeseries)
#     list_past_cov_scaled.append(scaled_past_cov_timeseries)
    # 159.89초

print(time.time() - start_time)

# Reset warning filter
warnings.filterwarnings('default')

0 217190_제너셈_20000101_20230331.csv
1 086670_비엠티_20000101_20230331.csv
2 271850_다이오진_20000101_20230331.csv
3 036200_유니셈_20000101_20230331.csv
4 302550_리메드_20000101_20230331.csv
5 053300_한국정보인증_20000101_20230331.csv
6 049830_승일_20000101_20230331.csv
7 006890_태경케미컬_20000101_20230331.csv
8 065710_서호전기_20000101_20230331.csv
9 013520_화승코퍼레이션_20000101_20230331.csv
10 054800_아이디스홀딩스_20000101_20230331.csv
11 199800_툴젠_20000101_20230331.csv
12 083790_크리스탈지노믹스_20000101_20230331.csv
13 005390_신성통상_20000101_20230331.csv
14 013310_아진산업_20000101_20230331.csv
15 005180_빙그레_20000101_20230331.csv
16 007700_F&F홀딩스_20000101_20230331.csv
17 251370_와이엠티_20000101_20230331.csv
18 004080_신흥_20000101_20230331.csv
19 192650_드림텍_20000101_20230331.csv
20 214270_FSN_20000101_20230331.csv
21 035290_골드앤에스_20000101_20230331.csv
22 288620_에스퓨얼셀_20000101_20230331.csv
23 442770_IBKS제21호스팩_20000101_20230331.csv
24 123890_한국자산신탁_20000101_20230331.csv
25 303530_이노뎁_20000101_20230331.csv
26 204620_글로벌텍스프리_20000101_20230331.c

238 163560_동일고무벨트_20000101_20230331.csv
239 003310_대주산업_20000101_20230331.csv
240 402030_코난테크놀로지_20000101_20230331.csv
241 33637L_솔루스첨단소재2우B_20000101_20230331.csv
242 016920_카스_20000101_20230331.csv
243 050890_쏠리드_20000101_20230331.csv
244 011000_진원생명과학_20000101_20230331.csv
245 139050_시티랩스_20000101_20230331.csv
246 277070_린드먼아시아_20000101_20230331.csv
247 365900_브이씨_20000101_20230331.csv
248 241770_메카로_20000101_20230331.csv
249 000725_현대건설우_20000101_20230331.csv
250 017390_서울가스_20000101_20230331.csv
251 399720_가온칩스_20000101_20230331.csv
252 155650_와이엠씨_20000101_20230331.csv
253 024880_케이피에프_20000101_20230331.csv
254 243870_굿센_20000101_20230331.csv
255 367480_유안타제8호스팩_20000101_20230331.csv
256 084370_유진테크_20000101_20230331.csv
257 009190_대양금속_20000101_20230331.csv
258 333430_일승_20000101_20230331.csv
259 299030_하나기술_20000101_20230331.csv
260 064800_젬백스링크_20000101_20230331.csv
261 018500_동원금속_20000101_20230331.csv
262 009780_엠에스씨_20000101_20230331.csv
263 036560_영풍정밀_20000101_20230331.csv

473 014285_금강공업우_20000101_20230331.csv
474 056360_코위버_20000101_20230331.csv
475 089980_상아프론테크_20000101_20230331.csv
476 115570_스타플렉스_20000101_20230331.csv
477 201490_미투온_20000101_20230331.csv
478 215090_휴센텍_20000101_20230331.csv
479 224060_더코디_20000101_20230331.csv
480 224760_엔에스컴퍼니_20000101_20230331.csv
481 322000_현대에너지솔루션_20000101_20230331.csv
482 001290_상상인증권_20000101_20230331.csv
483 086960_MDS테크_20000101_20230331.csv
484 003830_대한화섬_20000101_20230331.csv
485 019210_와이지-원_20000101_20230331.csv
486 114120_크루셜텍_20000101_20230331.csv
487 131400_이브이첨단소재_20000101_20230331.csv
488 109080_옵티시스_20000101_20230331.csv
489 042700_한미반도체_20000101_20230331.csv
490 053580_웹케시_20000101_20230331.csv
491 002020_코오롱_20000101_20230331.csv
492 238500_로보쓰리_20000101_20230331.csv
493 339950_아이비김영_20000101_20230331.csv
494 080220_제주반도체_20000101_20230331.csv
495 143210_핸즈코퍼레이션_20000101_20230331.csv
496 007770_한일화학_20000101_20230331.csv
497 039200_오스코텍_20000101_20230331.csv
498 180400_DXVX_20000101_20230331.

696 067570_엔브이에이치코리아_20000101_20230331.csv
697 033240_자화전자_20000101_20230331.csv
698 186230_그린플러스_20000101_20230331.csv
699 193250_와이제이엠게임즈_20000101_20230331.csv
700 323280_태성_20000101_20230331.csv
701 064290_인텍플러스_20000101_20230331.csv
702 222420_쎄노텍_20000101_20230331.csv
703 035810_이지홀딩스_20000101_20230331.csv
704 123330_제닉_20000101_20230331.csv
705 052300_초록뱀컴퍼니_20000101_20230331.csv
706 088790_진도_20000101_20230331.csv
707 018670_SK가스_20000101_20230331.csv
708 238170_엔에스엠_20000101_20230331.csv
709 023440_제이스코홀딩스_20000101_20230331.csv
710 115960_연우_20000101_20230331.csv
711 445360_비엔케이제1호스팩_20000101_20230331.csv
712 004140_동방_20000101_20230331.csv
713 000440_중앙에너비스_20000101_20230331.csv
714 338100_NH프라임리츠_20000101_20230331.csv
715 049470_SGA_20000101_20230331.csv
716 339770_교촌에프앤비_20000101_20230331.csv
717 008060_대덕_20000101_20230331.csv
718 298050_효성첨단소재_20000101_20230331.csv
719 093510_엔지브이아이_20000101_20230331.csv
720 195990_에이비프로바이오_20000101_20230331.csv
721 039310_세중_20000101_2023

927 252500_세화피앤씨_20000101_20230331.csv
928 105840_우진_20000101_20230331.csv
929 110020_전진바이오팜_20000101_20230331.csv
930 020400_대동금속_20000101_20230331.csv
931 047820_초록뱀미디어_20000101_20230331.csv
932 217950_파마리서치바이오_20000101_20230331.csv
933 044820_코스맥스비티아이_20000101_20230331.csv
934 000050_경방_20000101_20230331.csv
935 041910_에스텍파마_20000101_20230331.csv
936 089890_코세스_20000101_20230331.csv
937 001270_부국증권_20000101_20230331.csv
938 036000_예림당_20000101_20230331.csv
939 149980_하이로닉_20000101_20230331.csv
940 003545_대신증권우_20000101_20230331.csv
941 060900_대한그린파워_20000101_20230331.csv
942 402340_SK스퀘어_20000101_20230331.csv
943 009290_광동제약_20000101_20230331.csv
944 044450_KSS해운_20000101_20230331.csv
945 136490_선진_20000101_20230331.csv
946 032560_황금에스티_20000101_20230331.csv
947 096870_엘디티_20000101_20230331.csv
948 086890_이수앱지스_20000101_20230331.csv
949 291650_압타머사이언스_20000101_20230331.csv
950 067290_JW신약_20000101_20230331.csv
951 111870_KH 전자_20000101_20230331.csv
952 112040_위메이드_20000101_20230331.

...

1778 013990_아가방컴퍼니_20000101_20230331.csv
1779 900300_오가닉티코스메틱_20000101_20230331.csv
1780 065450_빅텍_20000101_20230331.csv
1781 002795_아모레G우_20000101_20230331.csv
1782 012160_영흥_20000101_20230331.csv
1783 438580_엔에이치스팩25호_20000101_20230331.csv
1784 010240_흥국_20000101_20230331.csv
1785 000500_가온전선_20000101_20230331.csv
1786 361610_SK아이이테크놀로지_20000101_20230331.csv
1787 003160_디아이_20000101_20230331.csv
1788 000210_DL_20000101_20230331.csv
1789 054670_대한뉴팜_20000101_20230331.csv
1790 314140_알피바이오_20000101_20230331.csv
1791 002380_KCC_20000101_20230331.csv
1792 297090_씨에스베어링_20000101_20230331.csv
1793 033340_좋은사람들_20000101_20230331.csv
1794 340570_티앤엘_20000101_20230331.csv
1795 084870_TBH글로벌_20000101_20230331.csv
1796 142210_유니트론텍_20000101_20230331.csv
1797 446070_유니드비티플러스_20000101_20230331.csv
1798 000490_대동_20000101_20230331.csv
1799 220260_켐트로스_20000101_20230331.csv
1800 092220_KEC_20000101_20230331.csv
1801 102950_아하_20000101_20230331.csv
1802 246710_티앤알바이오팹_20000101_20230331.csv
1803 1518

2000 256150_한독크린텍_20000101_20230331.csv
2001 162120_루켄테크놀러지스_20000101_20230331.csv
2002 276040_스코넥_20000101_20230331.csv
2003 021040_대호특수강_20000101_20230331.csv
2004 204610_티쓰리_20000101_20230331.csv
2005 065660_안트로젠_20000101_20230331.csv
2006 198940_한주라이트메탈_20000101_20230331.csv
2007 101400_엔시트론_20000101_20230331.csv
2008 036120_SCI평가정보_20000101_20230331.csv
2009 068790_DMS_20000101_20230331.csv
2010 223310_커머스마이너_20000101_20230331.csv
2011 058970_엠로_20000101_20230331.csv
2012 332570_와이팜_20000101_20230331.csv
2013 065170_비엘팜텍_20000101_20230331.csv
2014 015360_예스코홀딩스_20000101_20230331.csv
2015 199150_데이터스트림즈_20000101_20230331.csv
2016 291810_핀텔_20000101_20230331.csv
2017 017250_인터엠_20000101_20230331.csv
2018 311060_엘에이티_20000101_20230331.csv
2019 245620_EDGC_20000101_20230331.csv
2020 055490_테이팩스_20000101_20230331.csv
2021 001450_현대해상_20000101_20230331.csv
2022 011155_CJ씨푸드1우_20000101_20230331.csv
2023 036180_지더블유바이텍_20000101_20230331.csv
2024 006200_한국전자홀딩스_20000101_20230331.csv
2025 2

2205 032640_LG유플러스_20000101_20230331.csv
2206 239610_에이치엘사이언스_20000101_20230331.csv
2207 131370_알서포트_20000101_20230331.csv
2208 036580_팜스코_20000101_20230331.csv
2209 007370_진양제약_20000101_20230331.csv
2210 037330_인지디스플레_20000101_20230331.csv
2211 950200_소마젠_20000101_20230331.csv
2212 207760_미스터블루_20000101_20230331.csv
2213 173130_오파스넷_20000101_20230331.csv
2214 089470_HDC현대EP_20000101_20230331.csv
2215 147830_제룡산업_20000101_20230331.csv
2216 042670_현대두산인프라코어_20000101_20230331.csv
2217 183190_아세아시멘트_20000101_20230331.csv
2218 020180_대신정보통신_20000101_20230331.csv
2219 089010_켐트로닉스_20000101_20230331.csv
2220 078150_HB테크놀러지_20000101_20230331.csv
2221 308080_바이젠셀_20000101_20230331.csv
2222 003547_대신증권2우B_20000101_20230331.csv
2223 336260_두산퓨얼셀_20000101_20230331.csv
2224 331920_셀레믹스_20000101_20230331.csv
2225 439410_엔에이치스팩26호_20000101_20230331.csv
2226 085910_네오티스_20000101_20230331.csv
2227 357580_아모센스_20000101_20230331.csv
2228 002720_국제약품_20000101_20230331.csv
2229 060590_씨티씨바이오_20000101_2023

2422 000080_하이트진로_20000101_20230331.csv
2423 150840_인트로메딕_20000101_20230331.csv
2424 019440_세아특수강_20000101_20230331.csv
2425 057680_티사이언티픽_20000101_20230331.csv
2426 225530_보광산업_20000101_20230331.csv
2427 007120_미래아이앤지_20000101_20230331.csv
2428 089790_제이티_20000101_20230331.csv
2429 013580_계룡건설_20000101_20230331.csv
2430 327260_RF머트리얼즈_20000101_20230331.csv
2431 126880_제이엔케이히터_20000101_20230331.csv
2432 355150_코스텍시스_20000101_20230331.csv
2433 302440_SK바이오사이언스_20000101_20230331.csv
2434 146320_비씨엔씨_20000101_20230331.csv
2435 286750_나노브릭_20000101_20230331.csv
2436 073190_듀오백_20000101_20230331.csv
2437 004020_현대제철_20000101_20230331.csv
2438 422040_엔에이치스팩23호_20000101_20230331.csv
2439 000720_현대건설_20000101_20230331.csv
2440 348350_위드텍_20000101_20230331.csv
2441 311690_CJ 바이오사이언스_20000101_20230331.csv
2442 000227_유유제약2우B_20000101_20230331.csv
2443 232530_이엠티_20000101_20230331.csv
2444 000885_한화우_20000101_20230331.csv
2445 322970_무진메디_20000101_20230331.csv
2446 030720_동원수산_20000101_20230331.c

2652 003080_성보화학_20000101_20230331.csv
2653 215200_메가스터디교육_20000101_20230331.csv
2654 009810_플레이그램_20000101_20230331.csv
2655 096760_JW홀딩스_20000101_20230331.csv
2656 382480_지아이텍_20000101_20230331.csv
2657 028300_HLB_20000101_20230331.csv
2658 001275_부국증권우_20000101_20230331.csv
2659 122830_원포유_20000101_20230331.csv
2660 047810_한국항공우주_20000101_20230331.csv
2661 092070_디엔에프_20000101_20230331.csv
2662 289010_아이스크림에듀_20000101_20230331.csv
2663 049430_코메론_20000101_20230331.csv
2664 091440_텔레필드_20000101_20230331.csv
2665 054040_한국컴퓨터_20000101_20230331.csv
2666 083470_이엠앤아이_20000101_20230331.csv
2667 119500_포메탈_20000101_20230331.csv
2668 376180_피코그램_20000101_20230331.csv
2669 004310_현대약품_20000101_20230331.csv
2670 281820_케이씨텍_20000101_20230331.csv
2671 304840_피플바이오_20000101_20230331.csv
2672 403550_쏘카_20000101_20230331.csv
2673 255440_야스_20000101_20230331.csv
2674 153460_네이블_20000101_20230331.csv
2675 189980_흥국에프엔비_20000101_20230331.csv
2676 158430_아톤_20000101_20230331.csv
2677 008970_동양철관_200

In [ ]:
interpolate_list = ['closing_price'] + past_cov_list

# 필터링할 날짜 범위 설정
start_date = pd.to_datetime('2015-01-01')
end_date = pd.to_datetime('2023-03-31')

list_stock_timeseries = []

list_train_series_scaled = []
list_val_series_scaled = []
list_scaler = []

list_past_cov_scaled = []
list_train_scaled_past_cov = []
list_val_scaled_past_cov = []

list_future_cov_scaled = []
list_train_scaled_future_cov = []
list_val_scaled_future_cov = []

for i, file_name in enumerate(os.listdir(base_path)):
    print(i, file_name)
    df = pd.read_csv(base_path + file_name).rename(columns=column_mapping)
    if len(df) < 100:
        continue
    df['date'] = pd.to_datetime(df['date'])
    
    # Date interpolate
    stock_timeseries_df_interpolated = df.copy()
    stock_timeseries_df_interpolated.set_index('date', inplace=True)
    stock_timeseries_df_interpolated = stock_timeseries_df_interpolated.asfreq('B')
    for col in interpolate_list:
        stock_timeseries_df_interpolated[col] = stock_timeseries_df_interpolated[col].interpolate(method='linear')
    stock_timeseries_df_interpolated.reset_index(inplace=True)
    df = stock_timeseries_df_interpolated
    
    # Interpolate 확인
#     print(df.isna().sum())

    timeseries = TimeSeries.from_dataframe(df, time_col='date', freq='B')
    
    
    # Darts TimeSeries 변환 및 날짜 필터링
    stock_timeseries = timeseries['closing_price']
    stock_timeseries = stock_timeseries[start_date:end_date]
    list_stock_timeseries.append(stock_timeseries)

    # Past covariant 및 날짜 필터링
    scaled_past_cov_timeseries = getScaledPastCov(timeseries)
    scaled_past_cov_timeseries = scaled_past_cov_timeseries[start_date:end_date]
    list_past_cov_scaled.append(scaled_past_cov_timeseries)
    
    # Split train/validation set
    train_percentage = 0.8
    split_index = int(train_percentage * len(stock_timeseries))
    
    train_series, val_series = stock_timeseries[:split_index], stock_timeseries[split_index:]
    train_past_cov_series, val_past_cov_series = scaled_past_cov_timeseries[:split_index], scaled_past_cov_timeseries[split_index:]
    
    list_train_scaled_past_cov.append(train_past_cov_series)
    list_val_scaled_past_cov.append(val_past_cov_series)
    
    # Scale
    scaler = Scaler()
    train_series_scaled = scaler.fit_transform(train_series)
    val_series_scaled = scaler.transform(val_series)
    list_scaler.append(scaler)
    
    list_train_series_scaled.append(train_series_scaled)
    list_val_series_scaled.append(val_series_scaled)
    
#     if i % 50 == 49:
#         break

In [ ]:
logger = TensorBoardLogger('logs', name='my_model')

# A TorchMetric or val_loss can be used as the monitor
torch_metrics = MetricCollection([MeanAbsoluteError(), MeanSquaredError()])

# Early stop callback
my_stopper = EarlyStopping(
    monitor="val_MeanAbsoluteError",  # "val_loss",
    patience=5,
    min_delta=0.05,
    mode='min',
)
pl_trainer_kwargs = {"callbacks": [my_stopper], "logger": logger}

model = NLinearModel(input_chunk_length=20, output_chunk_length=5, n_epochs=10, random_state=42,
                        torch_metrics=torch_metrics,
                        pl_trainer_kwargs=pl_trainer_kwargs)
model.fit(list_train_series_scaled,
#             past_covariates=list_train_scaled_past_cov,
            # future_covariates=list_train_scaled_future_cov
            val_series=list_val_series_scaled,
#             val_past_covariates=list_val_scaled_past_cov,
            # val_future_covariates=list_val_scaled_future_cov
            )

In [7]:
model_path = "model/all_past.pkl"
if not os.path.exists('model'):
    os.mkdir('model')
model.save(model_path)
model_loaded = NLinearModel.load(model_path)

NotImplementedError: ('{} cannot be pickled', '_SingleProcessDataLoaderIter')

In [ ]:
# 1. past_covariates를 아무것도 주지 않았을 경우 상승, 하강이 그럴싸하게 나옴. 전부를 주었을 경우 하강만 나옴.
i = 21
predicted_series_scaled = model.predict(n=len(list_val_series_scaled[i]),
                                        series=list_train_series_scaled[i],
#                                         past_covariates=list_past_cov_scaled[i],
                                        # future_covariates=list_future_cov_scaled[i],
                                        )
prediction = list_scaler[i].inverse_transform(predicted_series_scaled)
list_stock_timeseries[i].plot()
prediction.plot(label="forecast")
plt.legend()

In [ ]:
prediction